In [1]:
#!conda activate huggingface_env
#conda info
#ls
# import packages
import pandas as pd
import numpy as np
import re
import spacy
import random
from sklearn.model_selection import train_test_split
pd.options.display.max_colwidth = 1000
#ls

In [2]:
#load annotated dataset
df = pd.read_excel('sentNeueClean7joined2_df.xlsx', sheet_name='Sheet1')


#keep only rows where 'Annotierung' is not null
df = df[df['Annotierung'].notnull()]
df.head(3)
#keep only columns 'name' 'vorname' 'Satz5' 'Annotierung'
df = df[['name', 'vorname', 'name_mutter', 'vorname_mutter', 'name_vater', 'vorname_vater', 'Satz5', 'Annotierung']]
#rename columns 'Annotierung' to 'label'
df = df.rename(columns={'Annotierung': 'Gewalt'})
#print number of rows and columns
print(df.shape)
#change type of column 'label' to int
df['Gewalt'] = df['Gewalt'].astype(int)
#print firts 3 rows
df.head(3)
#drop rows where 'label' is -1
df = df[df['Gewalt'] != -1]

(1006, 8)


# Consider Negation Outlier Corrections

In [3]:
# aussnegkorr_df = pd.read_excel('aussreisser_negation_20230614_2.xlsx', sheet_name='Sheet1')
#neue  version von aussreisser_negation_20230614_2.xlsx mit David verglichen
aussnegkorr_df = pd.read_excel('aussreisser_negation_20230614_2_anDavid_4_vergleich_Dragan_besprochen.xlsx', sheet_name='Sheet1')
print(list(aussnegkorr_df.columns))


['sentence', 'label', 'label laet', 'predicted_label', 'loss', 'wrong', 'Bemerkungen', 'korrektur_label', 'korr_lab_dichot', 'diff_stdg_laet', 'Korr_besprochen']


In [4]:

#keep only columns sentence and korrektur_label
aussnegkorr_df = aussnegkorr_df[['sentence', 'korrektur_label', 'Korr_besprochen']]

#create a new column 'korrektur_label2' and fill it with Korr_besprochen if Korr_besprochen is not null, else fill it with korrektur_label
aussnegkorr_df['korrektur_label2'] = np.where(aussnegkorr_df['Korr_besprochen'].notnull(), aussnegkorr_df['Korr_besprochen'], aussnegkorr_df['korrektur_label'])
#rename korrektur_label to korrektur_label_alt
aussnegkorr_df = aussnegkorr_df.rename(columns={'korrektur_label': 'korrektur_label_alt'})
#rename korrektur_label2 to korrektur_label
aussnegkorr_df = aussnegkorr_df.rename(columns={'korrektur_label2': 'korrektur_label'})
#drop column Korr_besprochen
aussnegkorr_df = aussnegkorr_df.drop(columns=['Korr_besprochen'])

#rename columns 'sentence' to 'Satz5'
aussnegkorr_df = aussnegkorr_df.rename(columns={'sentence': 'Satz5'})
print(list(aussnegkorr_df.columns))


['Satz5', 'korrektur_label_alt', 'korrektur_label']


In [5]:

#join df and aussnegkorr_df on column 'sentence'
df = df.join(aussnegkorr_df.set_index('Satz5'), on='Satz5')
#show counts of all columns
df.count()
#print all Satz5 Gewalt korrektur_label columns where korrektur_label is not null
# df[df['korrektur_label'].notnull()][['Satz5', 'Gewalt', 'korrektur_label']]
#replace Gewalt with korrektur_label where korrektur_label is not null
df.loc[df['korrektur_label'].notnull(), 'Gewalt'] = df['korrektur_label']
# df[df['korrektur_label'].notnull()][['Satz5', 'Gewalt', 'korrektur_label']]

# Replace pronouns with the names in the sentences 


In [6]:
nlp = spacy.load("de_core_news_lg")
# doc = nlp("Der alte Vater hat jungen Jason nicht geschlagen und auch nicht misshandelt, es war ein schöner Tag.")
# # doc = nlp("Berlin sieht wie eine schöne Stadt aus")
# print(doc)

# for token in doc:
#     print("1", token.lemma_, token.pos_, token.morph)

# for chunk in doc.noun_chunks:
#     print("2", chunk.text, chunk.root.text, chunk.root.dep_,
#             chunk.root.head.text)

# for ent in doc.ents:
#     print("3", ent.text, ent.start_char, ent.end_char, ent.label_)
# man muss den main_init mit den localen pfad vom de zip file ergänzen, siehe lib spacy vorlage
# !python -m pip install coreferee
# !python -m coreferee install de
import coreferee
# nlp = spacy.load('de_core_news_')
nlp.add_pipe('coreferee')

# correference resolution function for pronouns in a sentence
def coreference_replacement(sentence):
    if isinstance(sentence, str):
        doc = nlp(sentence)
        for x in range(len(doc._.coref_chains)):

            for y in range(len(doc._.coref_chains[x])):

                for z in range(len(doc._.coref_chains[x][y])):

                    int=doc._.coref_chains[x][y][z]
    
                    replace_with=str(doc._.coref_chains.resolve(doc[int]))
                    replace_with=replace_with.replace("["," ")
                    replace_with=replace_with.replace("]"," ")
                    replace_with_strip=replace_with.strip()
                    if replace_with=='None':

                        pass
                    else:
                        for ent in doc.ents:
                            
                            if ent.text==replace_with_strip and (ent.label_=="PER" or ent.label_=="MISC"):
                                if str(doc[doc._.coref_chains[x][y][z]]).lower() == "er" or str(doc[doc._.coref_chains[x][y][z]]).lower() == "sie":
                                    sentence=sentence.replace(" "+str(doc[doc._.coref_chains[x][y][z]])+" ", replace_with)
#                                     print(doc[doc._.coref_chains[x][y][z]], doc[doc._.coref_chains[x][y][z]].pos_, replace_with)
        return sentence
    else: 
        return sentence
    
# create a new index column 'index' and set it as index
df['index'] = df.index
# repare this error " # If we have a listlike key, _check_indexing_error will raise KeyError: 1"
df = df.reset_index(drop=True)
# for i in range(100):    
#      df.loc[i, "Satz5_prep"] = coreference_replacement(df.loc[i, "Satz5"])
# for i in range(100):    
#     df.loc[i, "Satz5_prep"] = coreference_replacement(df.loc[i, "Satz5"])  

# run the coreference_replacement function on all rows in column 'Satz5' and save the result in column 'Satz5_prep'
for i in range(len(df)):    
    df.loc[i, "Satz5_prep"] = coreference_replacement(df.loc[i, "Satz5"])  
    

#print Satz5_prep and Satz5 for the first 30 rows
# df[['Satz5_prep', 'Satz5']].head(30)
# show me different strings between Satz5_prep and Satz5 for the first 30 rows
# df[df['Satz5_prep'] != df['Satz5']][['Satz5_prep', 'Satz5']].head(30)
# df_sample=df[:100]
# df_sample[['s51', 's52', 's53', 's54', 's55', 's56']] = df_sample["Satz5_prep"].str.split('|', 5, expand=True)
# df_sample[['s51', 's52', 'Satz', 's53', 's54', 's55', 's56', 'Satz5_prep']][:10]

# df[['s51', 's52', 's53', 's54', 's55', 's56']] = df["Satz5_prep"].str.split('|', 5, expand=True)


# Replace the names of the client, mother of the child, and father with their labels (CLIENT, CM, CF)

In [17]:
def similarity_replacement(sentence, keyword, replacement, similarity_rate=1):
    if isinstance(keyword, str):
        doc = nlp(sentence.lower())
        doc1 = nlp(sentence)

        doc_keyword=nlp(keyword.lower())

        token2=doc_keyword[0]
        for i in range(len(doc)):
            token1=doc[i]
            similarity2 = token1.similarity(token2)
            if similarity2 >= similarity_rate:
                sentence = sentence.replace(doc1[i].text, replacement)            
        return sentence        
    else: 
        return sentence
        
# for i in range(100):
#     df.loc[i, "Satz5_prep"] = similarity_replacement(df.loc[i, "s53"], 
#     df.loc[i, "VORNAME"], "KLIENT", similarity_rate=0.9)

# for i in range(100):    
#     df.loc[i, "Satz5_prep"] = similarity_replacement(df.loc[i, "Satz5_prep"], 
#     df.loc[i, "NAME_MUTTER"], "KINDSMUTTER", similarity_rate=0.9)
    
# for i in range(100):    
#     df.loc[i, "Satz5_prep"] = similarity_replacement(df.loc[i, "Satz5_prep"], 
#     df.loc[i, "NAME_VATER"] , "KINDSVATER", similarity_rate=0.9)  

# for i in range(100):    
#     df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#     "Frau KINDSVATER" , "KINDSMUTTER"))  
#     df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#     "Herr KINDSMUTTER" , "KINDSVATER"))  
#     df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#     "Herrn KINDSMUTTER" , "KINDSVATER"))  
#     df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#     "Herr KINDSVATER" , "KINDSVATER"))  
#     df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#     "Herrn KINDSVATER" , "KINDSVATER"))  
#     df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#     "Frau KINDSMUTTER" , "KINDSMUTTER"))  

In [19]:
# for i in range(len(df)):
# for i in range(100):
#     print("Input:", df.loc[i, "Satz"],"\n",
#           "Client:", df.loc[i, "VORNAME"], "\n",
#           "Output:", client_similarity_check(df.loc[i, "Satz"], df.loc[i, "VORNAME"]))
for i in range(len(df)):
    df.loc[i, "Satz5_prep"] = similarity_replacement(df.loc[i, "Satz5_prep"], 
    df.loc[i, "vorname"], "KLIENT", similarity_rate=0.8)

for i in range(len(df)):    
    df.loc[i, "Satz5_prep"] = similarity_replacement(df.loc[i, "Satz5_prep"], 
    df.loc[i, "name_mutter"], "KINDSMUTTER", similarity_rate=0.9)
    
for i in range(len(df)):    
    df.loc[i, "Satz5_prep"] = similarity_replacement(df.loc[i, "Satz5_prep"], 
    df.loc[i, "name_vater"] , "KINDSVATER", similarity_rate=0.9)  
#df[df['Satz5_prep'] != df['Satz5']][['Satz5_prep', 'Satz5']].head(30)
# for i in range(len(df)):
#     df.loc[i, "Satz5_prep"] = similarity_replacement(df.loc[i, "Satz5_prep"], 
#     df.loc[i, "VORNAME"], "KLIENT", similarity_rate=0.9)

# for i in range(len(df)):    
#     df.loc[i, "Satz5_prep"] = similarity_replacement(df.loc[i, "Satz5_prep"], 
#     df.loc[i, "NAME_MUTTER"], "KINDSMUTTER", similarity_rate=0.9)
    
# for i in range(len(df)):    
#     df.loc[i, "Satz5_prep"] = similarity_replacement(df.loc[i, "Satz5_prep"], 
#     df.loc[i, "NAME_VATER"] , "KINDSVATER", similarity_rate=0.9) 
# cleanup the Satz5_prep column from the following strings
for i in range(len(df)):
    if isinstance(df.loc[i, "Satz5_prep"], str):
        df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
        "Frau KINDSVATER" , "KINDSMUTTER"))  
        df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
        "Herr KINDSMUTTER" , "KINDSVATER"))  
        df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
        "Herrn KINDSMUTTER" , "KINDSVATER"))  
        df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
        "Herr KINDSVATER" , "KINDSVATER"))  
        df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
        "Herrn KINDSVATER" , "KINDSVATER"))  
        df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
        "Frau KINDSMUTTER" , "KINDSMUTTER"))  
# for i in range(len(df)):
#     if isinstance(df.loc[i, "Satz5_prep"], str):
#         df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#         "Frau KINDSVATER" , "KINDSMUTTER"))  
#         df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#         "Herr KINDSMUTTER" , "KINDSVATER"))  
#         df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#         "Herrn KINDSMUTTER" , "KINDSVATER"))  
#         df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#         "Herr KINDSVATER" , "KINDSVATER"))  
#         df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#         "Herrn KINDSVATER" , "KINDSVATER"))  
#         df.loc[i, "Satz5_prep"] = (df.loc[i, "Satz5_prep"].replace( 
#         "Frau KINDSMUTTER" , "KINDSMUTTER"))  
# df[['Satz5', 'Satz5_prep', 'vorname', 'name_mutter', 'name_vater']][300:330]

<ipython-input-17-ad5d94f1384c>:11: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity2 = token1.similarity(token2)


# Create a test dataset

In [25]:

data3=df.copy()
data3 = data3[data3.Gewalt != -1]
pd.crosstab(index=data3['Gewalt'], columns='count')
#drop rows with Gewalt equal to -1


col_0,count
Gewalt,
0,201
1,90
2,94
3,184
4,328
5,90


## Previously created test dataset is filtered out.

In [26]:
#read the excel tabel Test_weitereBegriffe_Annotiert_20230607.xlsx and save it as a dataframe
data4 = pd.read_excel("Test_weitereBegriffe_Annotiert_20230607.xlsx", sheet_name="Sheet1")
#keep only the row Satz5
data4 = data4[['Satz5']]
#create column drop and set it to 1
data4['drop'] = 1
#merge data3 and data4 on Satz5
data3 = pd.merge(data3, data4, on='Satz5', how='left')



#keep only the rows with drop is equal to null
data3 = data3[data3['drop'].isnull()]




#create a dataframe with name train from data3
train = data3.copy()
pd.crosstab(index=data3['Gewalt'], columns='count')

In [28]:
# 0 labels
!stop do this only once
# label_0=data3['Gewalt']==0
# data3_0=data3[label_0]
# train_0, test_0 = train_test_split(data3_0, test_size=50, random_state=42)
# # 1 labels
# label_1=data3['Gewalt']==1
# data3_1=data3[label_1]
# train_1, test_1 = train_test_split(data3_1, test_size=25, random_state=42)
# # 2 labels
# label_2=data3['Gewalt']==2
# data3_2=data3[label_2]
# train_2, test_2 = train_test_split(data3_2, test_size=25, random_state=42)
# # 3 labels
# label_3=data3['Gewalt']==3
# data3_3=data3[label_3]
# train_3, test_3 = train_test_split(data3_3, test_size=50, random_state=42)
# # 4 labels
# label_4=data3['Gewalt']==4
# data3_4=data3[label_4]
# train_4, test_4 = train_test_split(data3_4, test_size=50, random_state=42)
# # 5 labels
# label_5=data3['Gewalt']==5
# data3_5=data3[label_5]
# train_5, test_5 = train_test_split(data3_5, test_size=25, random_state=42)



# train = pd.concat([train_0, train_1, train_2, train_3, train_4, train_5], axis=0)
# test = pd.concat([test_0, test_1, test_2, test_3, test_4, test_5], axis=0)
# print(pd.crosstab(index=train['Gewalt'], columns='count'))
# print(pd.crosstab(index=test['Gewalt'], columns='count'))
# # !pip install openpyxl


# test.to_excel('Test_weitereBegriffe_Annotiert_20230607.xlsx',index=False)
# from pathlib import Path  
# filepath = Path('Test_weitereBegriffe_Annotiert_20230607.csv')  
# filepath.parent.mkdir(parents=True, exist_ok=True)  
# test.to_csv(filepath)  

Der Befehl "stop" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


# Export of 5 sentences that do not linguistically involve violence, dichotomous classification

In [29]:
# pd.crosstab(index=df['anotiert'], columns='count')
# anotiert =  df3['anotiert']==1.0 
gewaltsaetze_anotiert_sprachlich = train.copy()


gewalt_num =     [0,1,2,3,4,5]
gewalt_num_rec = [1,1,1,1,0,1]
gewalt_string = ["Satz hat sprachlich nicht mit Gewalt zu tun", "Satz hat sprachlich mit Gewalt zu tun"]
gewaltsaetze_anotiert_sprachlich['Gewalt'] = gewaltsaetze_anotiert_sprachlich['Gewalt'].replace(gewalt_num, gewalt_num_rec)

# data0 = gewaltsaetze_anotiert_sprachlich[['Satz5_prep', 'Satz5', 'Satz5_prep', 'Gewalt']].copy()
# temp_df= data0.copy()
# temp_df['lenght'] = temp_df['Satz'].str.split().apply(len)
# drop_50 =  temp_df['lenght']<=60
# temp_df = temp_df[drop_50]

data0 = gewaltsaetze_anotiert_sprachlich[['Satz5', 'Satz5_prep', 'Gewalt']].copy()
data1=data0.rename(columns={"Satz5": "sentence5", "Satz5_prep": "sentence5_prep", "Gewalt": "label"}).copy()
from pathlib import Path  
filepath = Path('temp5_sprachlich_weitereBegriffe_20230607.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data1.to_csv(filepath)  
data1 = pd.read_csv('temp5_sprachlich_weitereBegriffe_20230607.csv',delimiter=',',encoding="utf")
# type(data1)
pd.crosstab(index=data1['label'], columns='count')
# from sklearn.utils import shuffle
# temp=shuffle(data1)
# temp[:50]

col_0,count
label,
0,278
1,484


# Dataset of violent sentences, linguistically annotated, split

In [30]:
# %pwd
# import the data
#temp1_sprachlich = pd.read_csv("20210517_Predictive_Chance_Modelling_KJH_zhaw\Arbeitsbereich_Dragan/temp1_sprachlich.csv",delimiter=',',encoding="utf")
temp1_sprachlich = pd.read_csv("temp5_sprachlich_weitereBegriffe_20230607.csv",delimiter=',',encoding="utf")
temp1_sprachlich=temp1_sprachlich.dropna()
# temp1_sprachlich[temp1_sprachlich["sentence"].isna()]

pd.crosstab(index=temp1_sprachlich['label'], columns='count')
from pathlib import Path  
#filepath = Path('20210517_Predictive_Chance_Modelling_KJH_zhaw\Arbeitsbereich_Dragan/gewalt_sprachlich_train_valid_datensatz_20230131.csv')  
filepath = Path('gewalt_sprachlich_train_valid_datensatz_wB_20230607.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
temp1_sprachlich.to_csv(filepath)  

# Filtering the annotated classes, recoding the classes - excluding class 4 "Sentence has no linguistic connection with.."


In [31]:
# pd.crosstab(index=df3['anotiert'], columns='count')
# anotiert =  df3['anotiert']==1.0 
gewaltsaetze_anotiert = train.copy()
#einsatz_prodoc = gewaltsaetze_anotiert['Nsatz']==1

#gewaltsaetze_anotiert_einsatz = gewaltsaetze_anotiert[einsatz_prodoc]
#gewaltsaetze_anotiert.head()
# pd.crosstab(index=gewaltsaetze_anotiert['anotiert'], columns='count')
#pd.crosstab(index=gewaltsaetze_anotiert_einsatz['Gewalt'], columns='count')
#gewalt_num = [0,1,2,3,4]
#gewalt_string = ["Klient hat keine Gewalt erfahren", "Klient hat Gewalt erfahren", "Klient hat häusliche Gewalt erlebt", "Klient hat Gewalt ausgeübt", "Satz hat sprachlich nicht mit Gewalt zu tun"]
# recodierung ohne 'satz hat sprachlich nicht mit gewalt zu tun' 
drop_4 = gewaltsaetze_anotiert['Gewalt']!=4
gewaltsaetze_anotiert = gewaltsaetze_anotiert[drop_4]
# drop_6 = gewaltsaetze_anotiert['Gewalt']!=6
# gewaltsaetze_anotiert = gewaltsaetze_anotiert[drop_6]

In [32]:

gewalt_num = [0,1,2,3,5]
gewalt_num_rec = [0,1,2,3,4]
gewalt_string = ["Klient hat keine Gewalt erfahren", "Klient hat Gewalt erfahren", "Klient hat häusliche Gewalt erlebt", "Klient hat Gewalt ausgeübt",  "Satz hat sprachlich nicht mit Gewalt zu tun", "Klient hat sexuelle Gewalt erlebt", "Klient hat sexuelle Gewalt ausgeübt"]
gewaltsaetze_anotiert['Gewalt'] = gewaltsaetze_anotiert['Gewalt'].replace(gewalt_num, gewalt_num_rec)
#gewaltsaetze_anotiert_einsatz.head()
#gewaltsaetze_anotiert_einsatz.head(600)
#gewaltsaetze_anotiert_einsatz['Satz_ohne_Gewalt'] = np.where(gewaltsaetze_anotiert_einsatz['Gewalt'] == 4, 1, 0)
pd.crosstab(index=gewaltsaetze_anotiert['Gewalt'], columns='count')


col_0,count
Gewalt,
0,151
1,65
2,69
3,134
4,65


## Creating keywords for CARP classification (Clue Reasoning Prompting)

In [33]:
# print(list(data3.columns))
#create a new empty column "clues" 
# gewaltsaetze_anotiert['clues'] = ""
#save  data3 df to excel file with the name "negations_datensatz_für_clues_erstellung_20230714.xlsx"
from pathlib import Path
filepath = Path('gewaltsaetze_anotiert_datensatz_für_clues_erstellung_20230714.xlsx')
filepath.parent.mkdir(parents=True, exist_ok=True)
gewaltsaetze_anotiert.to_excel(filepath)

#import the sheet1 from excel file "gewaltsaetze_anotiert_datensatz_für_clues_erstellung_20230714_2.xlsx" and save it as dataframe gewaltsaetze_anotiert_clues
gewaltsaetze_anotiert_clues = pd.read_excel("gewaltsaetze_anotiert_datensatz_für_clues_erstellung_20230714_2.xlsx", sheet_name="Sheet1")
#keep only columns Satz5, clues, reason and only rows with clues not empty
gewaltsaetze_anotiert_clues = gewaltsaetze_anotiert_clues[['Satz5', 'clues', 'reason']].copy()
gewaltsaetze_anotiert_clues = gewaltsaetze_anotiert_clues[gewaltsaetze_anotiert_clues['clues'].notna()]
gewaltsaetze_anotiert_clues = gewaltsaetze_anotiert_clues[gewaltsaetze_anotiert_clues['reason'].notna()]
#count number of rows
# gewaltsaetze_anotiert_clues.shape
#drop the column clues from gewaltsaetze_anotiert
# gewaltsaetze_anotiert = gewaltsaetze_anotiert.drop(columns=['clues'])
#merge gewaltsaetze_anotiert and gewaltsaetze_anotiert_clues on Satz5
gewaltsaetze_anotiert2 = pd.merge(gewaltsaetze_anotiert, gewaltsaetze_anotiert_clues, left_on='Satz5', right_on='Satz5', how='left')
# gewaltsaetze_anotiert.head(100)


# Export 5 Sätze Dataset temp5.csv

In [43]:
#update recodierung der Gewaltklassen nach der Subklassifizierung

# print(gewaltsaetze_anotiert2.columns)
#print Gewalt column counts
# print(gewaltsaetze_anotiert2['Gewalt'].value_counts())
#read traindata_gewaltklassen_3bgrf_recoded_20230817.xlsx and save it a dataframe called gewaltklassen_recoded_subklassifiziert
gewaltklassen_recoded_subklassifiziert = pd.read_excel('traindata_gewaltklassen_wbgrf_recoded_20230817.xlsx')
#join gewaltklassen_recoded_subklassifiziert with gewaltsaetze_anotiert2 on the column 'Satz5' and save it as a new dataframe called gewaltsaetze_anotiert3
gewaltsaetze_anotiert3 = gewaltsaetze_anotiert2.join(gewaltklassen_recoded_subklassifiziert.set_index('Satz5'), on='Satz5')
# print(gewaltsaetze_anotiert3.columns)
# print(gewaltsaetze_anotiert3.shape)
print(gewaltsaetze_anotiert3['Gewalt'].value_counts())
print(gewaltsaetze_anotiert3['Gewalt_recoded'].value_counts())
#if Gewalt_recoded is 1 then recode Gewalt to 1
gewaltsaetze_anotiert3.loc[gewaltsaetze_anotiert3['Gewalt_recoded'] == 1, 'Gewalt'] = 1
#if Gewalt_recoded is 2 then recode Gewalt to 2
gewaltsaetze_anotiert3.loc[gewaltsaetze_anotiert3['Gewalt_recoded'] == 2, 'Gewalt'] = 2
#if Gewalt_recoded is 3 then recode Gewalt to 3
gewaltsaetze_anotiert3.loc[gewaltsaetze_anotiert3['Gewalt_recoded'] == 3, 'Gewalt'] = 3

#if Gewalt_recoded is 5 then recode Gewalt to 4
gewaltsaetze_anotiert3.loc[gewaltsaetze_anotiert3['Gewalt_recoded'] == 5, 'Gewalt'] = 4

print(gewaltsaetze_anotiert3['Gewalt'].value_counts())


data0 = gewaltsaetze_anotiert3[['Satz5', 'Satz5_prep', 'Gewalt', 'clues', 'reason']].copy()
data1=data0.rename(columns={"Satz5": "sentence5", "Satz5_prep": "sentence5_prep", "Gewalt": "label"}).copy()
from pathlib import Path  
filepath = Path('temp5_20230607.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data1.to_csv(filepath)  
data5 = pd.read_csv('temp5_20230607.csv',delimiter=',',encoding="utf")
type(data5)
#counnt number of rows
print(data5.shape)
#list column names
print(list(data5.columns))

0    151
3    134
2     69
4     65
1     65
Name: Gewalt, dtype: int64
3.0    13
2.0     9
5.0     5
1.0     2
Name: Gewalt_recoded, dtype: int64
3    147
0    122
2     78
4     70
1     67
Name: Gewalt, dtype: int64


# Dataset Violent Sentences, annotated with negation/violence, split

In [45]:
from pathlib import Path    
#data3 = pd.read_csv('20210517_Predictive_Chance_Modelling_KJH_zhaw\Arbeitsbereich_Dragan/temp1.csv',delimiter=',',encoding="utf")
data3 = pd.read_csv('temp5_20230607.csv',delimiter=',',encoding="utf")
# data3=data3.dropna('label')
# type(data3)
# data3[data3["sentence"].isna()]
gewalt_num = [1,2,3, 4]
gewalt_num_rec = [1,1,1, 1]
data3['label'] = data3['label'].replace(gewalt_num, gewalt_num_rec)
pd.crosstab(index=data3['label'], columns='count')


col_0,count
label,
0,122
1,362


In [46]:


data3 = data3.sample(frac=1).reset_index(drop=True)
from pathlib import Path  
#filepath = Path('20210517_Predictive_Chance_Modelling_KJH_zhaw\Arbeitsbereich_Dragan/gewalt_negation_train_valid_datensatz_20230131.csv')  
filepath = Path('gewalt_negation_train_valid_datensatz_wB_20230607.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data3.to_csv(filepath)  

# Dataset Violence Sentences, annotated  only violence, split

In [47]:
from pathlib import Path    
#data3 = pd.read_csv('20210517_Predictive_Chance_Modelling_KJH_zhaw\Arbeitsbereich_Dragan/temp1.csv',delimiter=',',encoding="utf")
data3 = pd.read_csv('temp5_20230607.csv',delimiter=',',encoding="utf")
# data3=data3.dropna()
# type(data3)
# data3[data3["sentence"].isna()]

drop_0=data3['label']!=0
data3=data3[drop_0]
gewalt_num = [1,2,3,4]
gewalt_num_rec = [0,1,2,3]
data3['label'] = data3['label'].replace(gewalt_num, gewalt_num_rec)
# print(pd.crosstab(index=data3['label'], columns='count'))



data3 = data3.sample(frac=1).reset_index(drop=True)
train_valid=data3.copy()
# pd.crosstab(index=train_valid['label'], columns='count')
from pathlib import Path  
filepath = Path('gewalt_5ersaetze_train_valid_datensatz_wB_20230607.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
train_valid.to_csv(filepath)  

# Encrypting Data

In [ ]:
# import required module
from cryptography.fernet import Fernet
import os

# key generation
# key = Fernet.generate_key()

# string the key in a file
# with open('filekey.key', 'wb') as filekey:
#     filekey.write(key)

# opening the key
keytemp=os.path.expanduser('filekey.key')
with open(keytemp, 'rb') as filekey:
	key = filekey.read()

print(key)

# using the generated key
fernet = Fernet(key)


In [49]:
# opening the original file to encrypt
with open('gewalt_5ersaetze_train_valid_datensatz_wB_20230607.csv', 'rb') as file:
	original = file.read()
	
# encrypting the file
encrypted = fernet.encrypt(original)

# opening the file in write mode and
# writing the encrypted data
with open('train5wbenc.csv', 'wb') as encrypted_file:
	encrypted_file.write(encrypted)

In [50]:

# opening the original file to encrypt
with open('gewalt_sprachlich_train_valid_datensatz_wB_20230607.csv', 'rb') as file:
	original = file.read()
	
# encrypting the file
encrypted = fernet.encrypt(original)

# opening the file in write mode and
# writing the encrypted data
with open('train5wb_sprachlich_enc.csv', 'wb') as encrypted_file:
	encrypted_file.write(encrypted)


In [51]:

# opening the original file to encrypt
with open('gewalt_negation_train_valid_datensatz_wB_20230607.csv', 'rb') as file:
	original = file.read()
	
# encrypting the file
encrypted = fernet.encrypt(original)

# opening the file in write mode and
# writing the encrypted data
with open('train5wb_negation_enc.csv', 'wb') as encrypted_file:
	encrypted_file.write(encrypted)
